# Feature Generation
This notebooks contains the feature engineering for the second model iteration.

# Imports

In [1]:
# Necessary to import custom modules
import os
from pathlib import Path
os.chdir("/home/jovyan/work")

import pandas as pd


from src.features import get_feature

# Processing
The overall processing consists of two steps:
1. get all features per team (e.g. wins as home team in 2016/2017), ...
2. get all games in the format team_home|team_guest|score_home|score_guest|home_win

### Features per team

In [2]:
df_avg_starter_pts = get_feature("avg_starter_score_preceeding_season")
df_avg_starter_pts.head()

,avg_pts,team,team_abbreviation,game,game_id,game_type,season,preceeding_season
0,865.50,Atlanta Hawks,ATL,ATL at HOU,34683,regular_season,2016/2017,2015/2016
1,865.50,Atlanta Hawks,ATL,UTA at ATL,34712,regular_season,2016/2017,2015/2016
2,865.50,Atlanta Hawks,ATL,TOR at ATL,34909,regular_season,2016/2017,2015/2016
3,968.75,Atlanta Hawks,ATL,HOU at ATL,34024,regular_season,2016/2017,2015/2016
4,861.00,Atlanta Hawks,ATL,ATL at ORL,34807,regular_season,2016/2017,2015/2016


### All games with score

In [49]:
df_games = get_feature("all_games_with_id")
df_games.head()

,game_id,season,team_home,team_guest,score_home,score_guest,home_win
0,43285,2017/2018,Boston Celtics,Brooklyn Nets,110,97,1
1,43253,2017/2018,Chicago Bulls,Brooklyn Nets,96,124,0
2,43238,2017/2018,Milwaukee Bucks,Brooklyn Nets,111,119,0
3,43216,2017/2018,Philadelphia 76ers,Brooklyn Nets,121,95,1
4,43201,2017/2018,Miami Heat,Brooklyn Nets,109,110,0


### Merging

In [50]:
# avg home points of preceeding year based on starter 
df_home = df_avg_starter_pts.drop(["game", "season", "preceeding_season", "team_abbreviation"], axis=1).merge(
    df_games.drop(["team_guest", "season", "score_home", "score_guest", "home_win"], axis=1),
    left_on=["game_id", "team"],
    right_on=["game_id", "team_home"]
).drop(["team_home"], axis=1)

# avg guest points of preceeding year based on starter 
df_guest = df_avg_starter_pts.drop(["game", "season", "preceeding_season", "team_abbreviation"], axis=1).merge(
    df_games.drop(["team_home", "season", "score_home", "score_guest", "home_win"], axis=1),
    left_on=["game_id", "team"],
    right_on=["game_id", "team_guest"]
).drop(["team_guest"], axis=1)

### Final merging

In [53]:
# final merging
df_final = df_home.merge(
    df_guest,
    left_on="game_id",
    right_on="game_id",
    suffixes=["_home", "_guest"]
).merge(
    df_games.drop(["team_home", "team_guest"], axis=1),
    left_on="game_id",
    right_on="game_id"
)[["season", "team_home", "team_guest", "avg_pts_home", "avg_pts_guest", "score_home", "score_guest", "home_win"]]
df_final.head()

,season,team_home,team_guest,avg_pts_home,avg_pts_guest,score_home,score_guest,home_win
0,2016/2017,Atlanta Hawks,Utah Jazz,865.50,873.25,95,120,0
1,2016/2017,Atlanta Hawks,Toronto Raptors,865.50,890.25,105,99,1
2,2016/2017,Atlanta Hawks,Houston Rockets,968.75,1229.75,112,97,1
3,2016/2017,Atlanta Hawks,Brooklyn Nets,602.50,834.00,92,107,0
4,2016/2017,Atlanta Hawks,Boston Celtics,818.00,1148.25,123,116,1


# Persistence

In [54]:
p = Path("./data/features")
if not p.exists():
    p.mkdir(parents=True)
df_final.to_csv("./data/features/second_iteration.csv", index=False)